In [72]:
import pandas as pd
import json

In [88]:
df = pd.read_csv('data/resultats.csv')

In [100]:
df['departement'] = df['departement'].str.pad(2, fillchar='0')
df['circo'] = df['circo'].map(str).str.pad(3,fillchar='0')
df['code'] = df['departement'] + df['circo']

df = df.set_index('code')

In [102]:
for c in df:
    if any(c.startswith(pref) for pref in ['Prénom', 'N°Panneau', 'Sexe']):
        del df[c]

In [103]:
noms = pd.concat([df[c] for c in df if c.startswith('Nom')], ignore_index=True).unique()

In [104]:
totaux = df[['departement_libelle', 'circo_libelle', 'inscrits', 'abstentions', 'votants', 'blancs', 'nuls', 'exprimes']].sort_index()

In [117]:
reste = df.iloc[:, 2+len(totaux.columns):]

In [118]:
res = []
for i in range(len(reste.columns)//2):
    colonnes = reste.iloc[:, 2*i:2*i+2]
    colonnes.columns = ['nom', 'votes']
    col = colonnes.reset_index().set_index(['code', 'nom'])['votes']
    res.append(col)

In [119]:
votes = pd.concat(res).unstack().sort_index()

In [138]:
exprimes = votes.divide(totaux['exprimes'], axis=0)

In [139]:
topology = json.load(open('data/circos_topo.json'))

In [142]:
for geometry in topology['objects']['circos']['geometries']:
    circo = geometry['properties']['ID']
    geometry['properties']['votes'] = exprimes.loc[circo].to_dict()

In [143]:
json.dump(topology, open('circos_exprimes.json', 'w'))